In [1]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import pymesh
import meshio

from mesh3d import Mesh3D

%matplotlib inline
plt.rcParams["figure.figsize"] = 12.8, 9.6

In [2]:
CELL_SIZE=1e-1

# Test mesh loading, saving & resizing

In [4]:
input_mesh = pymesh.load_mesh("meshes/sausage_shape.obj")

mesh = Mesh3D(input_mesh)

mesh.resize_input_mesh(CELL_SIZE)

pymesh.save_mesh("meshes/regularised_cell.ply", mesh.input_mesh)
mesh.write_input_mesh("meshes/regularised_cell.vtk")

# mesh.plot_mesh()

# Test tetrahedralisation

In [5]:
mesh.tetrahedralise(CELL_SIZE)
mesh.write_mesh("meshes/regularised_cell_tetra.vtk")

# Test boundary node marking

In [8]:
tetmesh = mesh.tetmesh
boundary_mesh = pymesh.compute_outer_hull(mesh.tetmesh)

is_boundary = np.zeros(tetmesh.num_vertices)
source_faces = boundary_mesh.get_attribute("face_sources").astype(int)
for i_face in source_faces:
    face = tetmesh.faces[i_face]
    is_boundary[face] = True

tetmesh.add_attribute("is_boundary")
tetmesh.set_attribute("is_boundary", is_boundary)

mesh.write_mesh("meshes/regularised_cell_tetra_with_boundary.vtk", ["is_boundary"])

# Test node data output

In [10]:
vertex_data = tetmesh.vertices[:, 2]
tetmesh.add_attribute("data")
tetmesh.set_attribute("data", vertex_data)

mesh.write_mesh("meshes/regularised_cell_tetra_with_data.vtk", ["data"])

# Test steady-state heat equation (i.e. stiffness matrix)

In [ ]:
def calculate_body_force(vertex):
    # Body force is simply f=4
    return 4.0

stiffness = mesh.assemble_stiffness()
force_vector = mesh.assemble_body_force(calculate_body_force)

# Apply boundary conditions
stiffness[mesh.boundary_vertices,:] = 0.0
stiffness[:,mesh.boundary_vertices] = 0.0
for vertex in mesh.boundary_vertices:
    stiffness[vertex, vertex] = 1.0
force_vector[mesh.boundary_vertices] = 0.0

# Solve
rhs_vector = force_vector
soln = np.linalg.solve(stiffness, rhs_vector)

# Calculate error
error = [
    abs(soln[i] - analytical_solution(vertex))
    for i, vertex in enumerate(mesh.vertices())]

print("Error is", np.linalg.norm(error))

# Plot
fig, ax = plt.subplots(1, 1, figsize=(10,10))
mesh.plot(soln, axis=ax)
ax.set_title('Solution')